# Data wrangling for covid data

## Import libraries

In [31]:
import pandas as pd
import numpy as np

## Load data from github

In [32]:
data = pd.read_csv(
    "https://raw.githubusercontent.com/coviddata/coviddata/master/data/sources/jhu_csse/standardized/standardized.csv"
)
data.date = pd.to_datetime(data.date)

In [33]:
data.head()

,date,country,region,place,cases,deaths,recoveries
0,2020-01-22,Mainland China,Anhui,NaN,1,0,0
1,2020-01-22,Mainland China,Beijing,NaN,14,0,0
2,2020-01-22,Mainland China,Chongqing,NaN,6,0,0
3,2020-01-22,Mainland China,Fujian,NaN,1,0,0
4,2020-01-22,Mainland China,Gansu,NaN,0,0,0


In [34]:
data.describe()

,cases,deaths,recoveries
count,48552.000000,48552.000000,48552.000000
mean,281.074930,12.204214,76.589656
std,3396.702516,240.281361,1544.602780
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,2.000000,0.000000,0.000000
75%,17.000000,0.000000,0.000000
max,119827.000000,14681.000000,63612.000000


In [35]:
df_world = data.drop(
    columns=['country', 'region', 'place']
).set_index(
    'date'
).resample(
    'd'
).sum(
    min_count=180
).reset_index(

).assign(
    country='World',
    region=np.nan,
    place=np.nan    
)
df_world

,date,cases,deaths,recoveries,country,region,place
0,2020-01-22,NaN,NaN,NaN,World,NaN,NaN
1,2020-01-23,NaN,NaN,NaN,World,NaN,NaN
2,2020-01-24,NaN,NaN,NaN,World,NaN,NaN
3,2020-01-25,NaN,NaN,NaN,World,NaN,NaN
4,2020-01-26,NaN,NaN,NaN,World,NaN,NaN
...,...,...,...,...,...,...,...
68,2020-03-30,782365.0,37582.0,164566.0,World,NaN,NaN
69,2020-03-31,857487.0,42107.0,178034.0,World,NaN,NaN
70,2020-04-01,932605.0,46809.0,193177.0,World,NaN,NaN
71,2020-04-02,1013157.0,52983.0,210263.0,World,NaN,NaN


In [36]:
df_world = pd.concat((data, df_world)).sort_values(['country', 'date'])
df_world.describe()

,cases,deaths,recoveries
count,4.858100e+04,48581.000000,48581.000000
mean,5.170431e+02,23.084086,140.260781
std,1.262515e+04,640.253240,3242.042092
min,0.000000e+00,0.000000,0.000000
25%,0.000000e+00,0.000000,0.000000
50%,2.000000e+00,0.000000,0.000000
75%,1.700000e+01,0.000000,0.000000
max,1.095917e+06,58787.000000,225796.000000


In [41]:
df_grouped = (
    df_world    
    .assign(confirmed_total=df_world.cases)
    .assign(recovered_total=df_world.recoveries)
    .assign(dead_total=df_world.deaths)
    .drop(columns=["cases", "recoveries", "deaths"])
    .assign(
        geo_key=df_world.country.str.cat(
            df_world.region.str.cat(
                df_world.place, 
                sep='_',
                na_rep='na'
            ), 
            sep='_',
            na_rep='na'
        )
    )
    .sort_values(["geo_key", "date"])
    .groupby(["geo_key"])
)
df_grouped.head()

,date,country,region,place,confirmed_total,recovered_total,dead_total,geo_key
8777,2020-03-22,US,Alabama,Autauga,0.0,0.0,0.0,US_Alabama_Autauga
11144,2020-03-23,US,Alabama,Autauga,0.0,0.0,0.0,US_Alabama_Autauga
14559,2020-03-24,US,Alabama,Autauga,1.0,0.0,0.0,US_Alabama_Autauga
17976,2020-03-25,US,Alabama,Autauga,4.0,0.0,0.0,US_Alabama_Autauga
21396,2020-03-26,US,Alabama,Autauga,6.0,0.0,0.0,US_Alabama_Autauga
...,...,...,...,...,...,...,...,...
12,2020-01-22,Hong Kong,Hong Kong,NaN,0.0,0.0,0.0,NaN
35,2020-01-22,Japan,NaN,NaN,2.0,0.0,0.0,NaN
20,2020-01-22,Macau,Macau,NaN,1.0,0.0,0.0,NaN
0,2020-01-22,Mainland China,Anhui,NaN,1.0,0.0,0.0,NaN


In [ ]:
df_new_column = (
    df_world    
    .assign(confirmed_total=df_world.cases)
    .assign(recovered_total=df_world.recoveries)
    .assign(dead_total=df_world.deaths)
    .sort_values(["country", "region", "place", "date"])
    .groupby(["country", "region", "place", "date"])
    .assign(
        confirmed_dayli=df_no_region.cases.subtract(
            df_no_region.confirmed.shift(1), fill_value=0
        ).clip(0)
    )
    .assign(
        recovered_dayli=df_no_region.recoveries.subtract(
            df_no_region.recovered.shift(1), fill_value=0
        ).clip(0)
    )
    .assign(
        dead_dayli=df_no_region.deaths.subtract(
            df_no_region.dead.shift(1), fill_value=0
        ).clip(0)
    )
    .reset_index()
    .drop(columns=["cases", "recoveries", "deaths"])
)
df_new_column.describe()